In [1]:
from lxml import etree
from pathlib import Path
from pathlib import PurePosixPath
from datetime import datetime
import os
import pandas as pd
import numpy as np
import pickle
import re
from pprint import pprint
from random import sample

from ddp_util import chatomid_to_url, url_to_chatomid

# load dfs

In [2]:
contents_full = pd.read_parquet('../../data/output/charters_2022-09-18-2211.parquet', engine='pyarrow')

In [3]:
thousand_charter_sample = pd.read_csv('../../data/in/1000charters_gsheet/1000charters_gsheet.csv', encoding='utf-8')

# builds lists

In [4]:
m_charters = thousand_charter_sample.iloc[:,2]

In [5]:
m_charters

0      https://www.monasterium.net/mom/HU-MNL-DLKINL/...
1      https://www.monasterium.net/mom/DE-BayHStA/Pas...
2      https://www.monasterium.net/mom/AT-StiAStams/U...
3      https://www.monasterium.net/mom/HU-MNL-DLEOKL/...
4      https://www.monasterium.net/mom/DE-HStAMa/urk7...
                             ...                        
995    https://www.monasterium.net/mom/AT-DOZA/Urkund...
996    https://www.monasterium.net/mom/HU-MNL-DLKINL/...
997    https://www.monasterium.net/mom/DE-HStAMa/urk7...
998    https://www.monasterium.net/mom/DE-BayHStA/KUM...
999    https://www.monasterium.net/mom/AT-StiAG/Goett...
Name: URL , Length: 1000, dtype: object

In [6]:
m_charters_list = list(m_charters)
m_charters_list_sample = sample(m_charters_list,10)

In [26]:
m_charters_ids = [url_to_chatomid(i) for i in m_charters_list]

# build difference

In [33]:
contents_full_exploded = contents_full.explode("atom_id")

In [37]:
charters_reduced = contents_full_exploded[~contents_full_exploded["atom_id"].isin(m_charters_ids)]

In [38]:
print(len(contents_full_exploded))
print(len(charters_reduced))

659170
658170


In [57]:
df_diff = pd.concat([charters_reduced.atom_id,contents_full_exploded.atom_id])

In [65]:
compttest = df_diff.drop_duplicates(keep=False).tolist()

In [68]:
compttest

['tag:www.monasterium.net,2011:/charter/AggOCart/1400_XI_04',
 'tag:www.monasterium.net,2011:/charter/ArdCan/1418_XI_25',
 'tag:www.monasterium.net,2011:/charter/ArdCan/1429_XII_21',
 'tag:www.monasterium.net,2011:/charter/AT-ADG/AGDK/2-5-3',
 'tag:www.monasterium.net,2011:/charter/AT-ADG/AGDK/97-23-6',
 'tag:www.monasterium.net,2011:/charter/AT-ADG/AGDK/X-3-7',
 'tag:www.monasterium.net,2011:/charter/AT-ADG/AGDK/Zweinitz_18',
 'tag:www.monasterium.net,2011:/charter/AT-AES/Urkunden/1080',
 'tag:www.monasterium.net,2011:/charter/AT-AES/Urkunden/1704',
 'tag:www.monasterium.net,2011:/charter/AT-AES/Urkunden/2217',
 'tag:www.monasterium.net,2011:/charter/AT-AES/Urkunden/3015',
 'tag:www.monasterium.net,2011:/charter/AT-AES/Urkunden/304',
 'tag:www.monasterium.net,2011:/charter/AT-AES/Urkunden/715',
 'tag:www.monasterium.net,2011:/charter/AT-AWMK/NeunkirchenOFMConv/9',
 'tag:www.monasterium.net,2011:/charter/AT-DAGS/Bistumsurkunden/I-187',
 'tag:www.monasterium.net,2011:/charter/AT-DAGS/Pf

# check if same

In [70]:
m_charters_ids.sort() == compttest.sort()

True

# generate new sample

In [74]:
newsample = charters_reduced.sample(n=4000)

In [81]:
contents_full[contents_full["atom_id"].astype(str).str.contains("charter/HU-")]

,atom_id,cei_abstract_joined,cei_abstract_foreign,cei_tenor_joined,cei_pTenor,cei_placeName,cei_lang_MOM,cei_date,cei_dateRange,cei_date_ATTRIBUTE_value,cei_dateRange_ATTRIBUTE_from,cei_dateRange_ATTRIBUTE_to,cei_graphic_ATTRIBUTE_url_orig,cei_graphic_ATTRIBUTE_url_copy
334497,"[tag:www.monasterium.net,2011:/charter/HU-BFL/...","V. Béla király elrendeli, hogy a budai/pesti m...",[],Ego,[],[],[],[],[],[],[],[],[],[]
334498,"[tag:www.monasterium.net,2011:/charter/HU-BFL/...",Buda város hatósága a német mészárosok céhének...,[],,[],[],[],[],[],[14810502],[],[],"[BFL_XV_5_3.jpg, BFL_XV_5_3_hatlap.jpg]",[]
334499,"[tag:www.monasterium.net,2011:/charter/HU-BFL/...",Buda város hatósága a német mészárosok céhének...,[],,[],[],[],[],[],[14820322],[],[],[BFL_XV_5_4.jpg],[]
334500,"[tag:www.monasterium.net,2011:/charter/HU-BFL/...",I. Mátyás király a német mészárosok céhének ve...,[],,[],[],[],[],[],[14840326],[],[],"[BFL_XV_5_5.jpg, BFL_XV_5_5_hatlap.jpg]",[]
334501,"[tag:www.monasterium.net,2011:/charter/HU-BFL/...",II. Ulászló király átírja és megerősíti Zsigmo...,[],,[],[],[],[],[],[14910620],[],[],"[BFL_XV_5_6.jpg, BFL_XV_5_6_hatlap.jpg]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502687,"[tag:www.monasterium.net,2011:/charter/HU-VFL/...",Aussteller: ?,[],,[],[],[],[],[],[15009999],[],[],[],[]
502688,"[tag:www.monasterium.net,2011:/charter/HU-VFL/...",Aussteller: ?,[],,[],[],[],[],[],[15009999],[],[],[],[]
502689,"[tag:www.monasterium.net,2011:/charter/HU-VFL/...",Aussteller: FEHÉRVÁRI KÁPTALAN,[],,[],[],[],[15040314],[],[15040314],[],[],[],[]
502690,"[tag:www.monasterium.net,2011:/charter/HU-VFL/...",Aussteller: GERGELY KAPORNAKI APÁT,[],,[],[],[],[],[],[15150116],[],[],[],[]


In [85]:
test = newsample[newsample["atom_id"].astype(str).str.contains("/HU-")]

In [92]:
test.atom_id.iloc[1]

'tag:www.monasterium.net,2011:/charter/HU-MNL-DLCSLP/FESCSQ73/159283'

In [82]:
thousand_charter_sample

,Unnamed: 0,Unnamed: 1,URL
0,NaN,NaN,https://www.monasterium.net/mom/HU-MNL-DLKINL/...
1,NaN,NaN,https://www.monasterium.net/mom/DE-BayHStA/Pas...
2,NaN,NaN,https://www.monasterium.net/mom/AT-StiAStams/U...
3,NaN,NaN,https://www.monasterium.net/mom/HU-MNL-DLEOKL/...
4,NaN,NaN,https://www.monasterium.net/mom/DE-HStAMa/urk7...
...,...,...,...
995,NaN,NaN,https://www.monasterium.net/mom/AT-DOZA/Urkund...
996,NaN,NaN,https://www.monasterium.net/mom/HU-MNL-DLKINL/...
997,NaN,NaN,https://www.monasterium.net/mom/DE-HStAMa/urk7...
998,NaN,NaN,https://www.monasterium.net/mom/DE-BayHStA/KUM...


In [93]:
chatomid_to_url("tag:www.monasterium.net,2011:/charter/HU-MNL-DLCSLP/FESCSQ73/159283")

'https://www.monasterium.net/mom/HU-MNL-DLCSLP/FESCSQ73/159283/charter'

In [84]:
thousand_charter_sample[thousand_charter_sample["URL"].astype(str).str.contains("/HU-")]

KeyError: 'URL'